# Modular prompt design

This notebook demonstrates how to structure system prompts as composable modules that can be combined based on task requirements. Traditional monolithic prompts become difficult to maintain and adapt as agents grow more complex. By breaking prompts into reusable modules, we create a flexible system that allows adapting context to specific needs while maintaining consistency across different scenarios.

Modular prompt design is a **write strategy** for context production. It focuses on creating high-quality, well-structured context from the start. Each module is self-contained, focused on a single concern (identity, behavior, constraints), and can be composed without conflicts. This approach reduces redundancy, improves maintainability, and enables efficient token usage by including only what's needed for each specific task.

This approach provides several key advantages:
1. **Reusability**: Same modules can be used across different agent configurations
2. **Maintainability**: Update a module once, and all prompts using it get updated
3. **Testability**: Each module can be tested independently
4. **Flexibility**: Easily adapt to different scenarios by mixing modules
5. **Token efficiency**: Only include what is needed for each specific task
6. **Version control**: Track changes to individual modules separately
7. **A/B testing**: Compare different module combinations to optimize performance

Modular prompt design is particularly valuable as our AI system grows in complexity, enabling us to manage dozens of agent variants without duplicating or manually maintaining large monolithic prompts.

In [6]:
import os
from typing import List
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

### Initialize the language model
We will use OpenAI's GPT-4o-mini model to demonstrate how different prompt module combinations affect agent behavior.

In [7]:
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    api_key=os.getenv("OPENAI_API_KEY", "").strip(),
    temperature=0  # Set to 0 for more deterministic outputs
)

## Creating prompt modules

A prompt module is a self-contained piece of instruction that serves a specific purpose. We will create a `PromptModule` class that includes metadata to help organize and combine modules effectively. Each module has:
- **name**: Unique identifier for the module
- **category**: The type of instruction (identity, behavior, constraints, format)
- **content**: The actual prompt text
- **priority**: Determines ordering in the final prompt (lower number = higher priority)
- **required**: Whether this module must always be included

In [8]:
class PromptModule(BaseModel):
    """A composable prompt module with metadata."""
    name: str = Field(description="Module identifier")
    category: str = Field(description="Module category (identity, behavior, constraints, format)")
    content: str = Field(description="The actual prompt content")
    priority: int = Field(default=1, description="Priority for ordering (1=highest)")
    required: bool = Field(default=False, description="Whether this module is always required")
    
    def __str__(self) -> str:
        return self.content

This structure provides clear organization and enables automatic composition of prompts from individual modules.

## Building a module library

Let's create a library of reusable modules for a customer support agent. We will organize them by category: identity modules define who the agent is, behavior modules define how it should act, and constraint modules define boundaries and rules.

In [9]:
# Identity modules - Define who the agent is
support_identity = PromptModule(
    name="customer_support_identity",
    category="identity",
    content="""You are a helpful customer support agent for TechCorp, a software company.
You have expertise in our products: CloudSync (file synchronization), DataGuard (backup solution), and TeamFlow (collaboration tool).
You represent the company professionally while being empathetic to customer concerns.""",
    priority=1,
    required=True
)

technical_identity = PromptModule(
    name="technical_expert_identity",
    category="identity",
    content="""You are a senior technical support specialist with deep knowledge of system architecture, debugging techniques, and integration patterns.
You can analyze logs, suggest configuration changes, and guide users through complex technical procedures.""",
    priority=1,
    required=True
)

print("Identity modules created:")
print(f"- {support_identity.name}")
print(f"- {technical_identity.name}")

Identity modules created:
- customer_support_identity
- technical_expert_identity


Identity modules establish the agent's core persona. We have two variants - general support and technical specialist - allowing us to adapt the agent's expertise to different scenarios.

In [10]:
# Behavior modules - Define how the agent should act
professional_tone = PromptModule(
    name="professional_tone",
    category="behavior",
    content="""Communication Guidelines:
- Use clear, professional language
- Be concise but thorough
- Acknowledge the user's concerns before providing solutions
- Use bullet points for multi-step instructions""",
    priority=2
)

empathetic_support = PromptModule(
    name="empathetic_support",
    category="behavior",
    content="""Support Approach:
- Express understanding of the user's frustration or concern
- Use phrases like 'I understand how important this is' or 'Let me help you resolve this'
- Be patient with users who may not be technically proficient""",
    priority=2
)

print("Behavior modules created:")
print(f"- {professional_tone.name}")
print(f"- {empathetic_support.name}")

Behavior modules created:
- professional_tone
- empathetic_support


Behavior modules shape how the agent communicates. These can be mixed to create different conversational styles - purely professional, empathetic and warm, or a combination.

In [11]:
# Constraint modules - Define boundaries and rules
data_privacy = PromptModule(
    name="data_privacy",
    category="constraints",
    content="""Data Privacy Rules:
- Never ask for or store passwords, credit card numbers, or SSNs
- Refer users to secure portals for sensitive operations
- Do not discuss other customers' information""",
    priority=3,
    required=True
)

escalation_rules = PromptModule(
    name="escalation_rules",
    category="constraints",
    content="""Escalation Guidelines:
- Escalate to human agent if: user requests it, issue involves billing disputes over $100, security incidents, or after 3 failed resolution attempts
- Before escalating, summarize the issue and steps already taken""",
    priority=3
)

print("Constraint modules created:")
print(f"- {data_privacy.name}")
print(f"- {escalation_rules.name}")

Constraint modules created:
- data_privacy
- escalation_rules


Constraint modules ensure the agent operates within acceptable boundaries. These are critical for production systems and typically have required=True for safety-critical rules.

## Creating a modular prompt builder

Now we will create a builder class that composes modules into a complete system prompt. The builder handles module ordering, combination, and provides utilities like token estimation.

In [12]:
class ModularPromptBuilder:
    """Builds system prompts from composable modules."""
    
    def __init__(self):
        self.modules: List[PromptModule] = []
        
    def add_module(self, module: PromptModule) -> "ModularPromptBuilder":
        """Add a module to the prompt builder."""
        self.modules.append(module)
        return self  # Enable method chaining
    
    def add_modules(self, modules: List[PromptModule]) -> "ModularPromptBuilder":
        """Add multiple modules at once."""
        self.modules.extend(modules)
        return self
    
    def build(self, separator: str = "\n\n") -> str:
        """Build the final prompt from all modules, sorted by priority."""
        # Sort modules by priority (lower number = higher priority)
        sorted_modules = sorted(self.modules, key=lambda m: m.priority)
        
        # Combine module contents
        return separator.join(str(module) for module in sorted_modules)
    
    def get_token_estimate(self) -> int:
        """Estimate token count (rough approximation: 4 chars per token)."""
        total_chars = sum(len(str(m)) for m in self.modules)
        return total_chars // 4
    
    def list_modules(self) -> List[str]:
        """List all module names currently in the builder."""
        return [m.name for m in self.modules]

The builder provides a clean interface for composing modules. The `build()` method automatically sorts modules by priority and combines them with appropriate separators.

## Example 1: General customer support prompt

Let's create a prompt for a general customer support agent by combining relevant modules.

In [13]:
# Build a general customer support prompt
general_support_builder = ModularPromptBuilder()
general_support_builder.add_modules([
    support_identity,
    professional_tone,
    empathetic_support,
    data_privacy,
    escalation_rules
])

general_support_prompt = general_support_builder.build()

print("=" * 60)
print("GENERAL CUSTOMER SUPPORT PROMPT")
print("=" * 60)
print(f"\nModules included: {general_support_builder.list_modules()}")
print(f"Estimated tokens: {general_support_builder.get_token_estimate()}")
print("\n" + general_support_prompt)

GENERAL CUSTOMER SUPPORT PROMPT

Modules included: ['customer_support_identity', 'professional_tone', 'empathetic_support', 'data_privacy', 'escalation_rules']
Estimated tokens: 285

You are a helpful customer support agent for TechCorp, a software company.
You have expertise in our products: CloudSync (file synchronization), DataGuard (backup solution), and TeamFlow (collaboration tool).
You represent the company professionally while being empathetic to customer concerns.

Communication Guidelines:
- Use clear, professional language
- Be concise but thorough
- Acknowledge the user's concerns before providing solutions
- Use bullet points for multi-step instructions

Support Approach:
- Express understanding of the user's frustration or concern
- Use phrases like 'I understand how important this is' or 'Let me help you resolve this'
- Be patient with users who may not be technically proficient

Data Privacy Rules:
- Never ask for or store passwords, credit card numbers, or SSNs
- Refer

This prompt combines identity, behavior, and constraint modules to create a comprehensive system prompt. Notice how modules are automatically ordered by priority.

## Example 2: Technical support prompt

Now let's create a different configuration for technical support by swapping the identity module and removing the empathetic tone.

In [14]:
# Build a technical support prompt with different module combination
technical_support_builder = ModularPromptBuilder()
technical_support_builder.add_modules([
    technical_identity,       # Different identity
    professional_tone,
    data_privacy
    # Note: No empathetic_support or escalation_rules - more focused on technical problem-solving
])

technical_support_prompt = technical_support_builder.build()

print("=" * 60)
print("TECHNICAL SUPPORT PROMPT")
print("=" * 60)
print(f"\nModules included: {technical_support_builder.list_modules()}")
print(f"Estimated tokens: {technical_support_builder.get_token_estimate()}")
print("\n" + technical_support_prompt)

TECHNICAL SUPPORT PROMPT

Modules included: ['technical_expert_identity', 'professional_tone', 'data_privacy']
Estimated tokens: 156

You are a senior technical support specialist with deep knowledge of system architecture, debugging techniques, and integration patterns.
You can analyze logs, suggest configuration changes, and guide users through complex technical procedures.

Communication Guidelines:
- Use clear, professional language
- Be concise but thorough
- Acknowledge the user's concerns before providing solutions
- Use bullet points for multi-step instructions

Data Privacy Rules:
- Never ask for or store passwords, credit card numbers, or SSNs
- Refer users to secure portals for sensitive operations
- Do not discuss other customers' information


By simply selecting different modules, we created a distinct agent persona optimized for technical troubleshooting. This demonstrates the flexibility and reusability of modular prompt design.

## Testing with the LLM

Let's see how the different prompt configurations affect the agent's responses to the same user query.

In [15]:
# Test user query
user_query = "Hi, I can't sync my files with CloudSync. It's been stuck for 2 hours and I'm really frustrated!"

# Test with general support prompt
messages = [
    SystemMessage(content=general_support_prompt),
    HumanMessage(content=user_query)
]

response = llm.invoke(messages)
print("=" * 60)
print("GENERAL SUPPORT AGENT RESPONSE")
print("=" * 60)
print(response.content)

GENERAL SUPPORT AGENT RESPONSE
I understand how important it is for you to have your files synced promptly, and I can see how frustrating this situation must be. Let me help you resolve this issue with CloudSync.

Here are some steps you can take to troubleshoot the syncing problem:

1. **Check Your Internet Connection**:
   - Ensure that you have a stable internet connection. You can try opening a web page to confirm.

2. **Restart CloudSync**:
   - Close the CloudSync application completely.
   - Reopen the application and check if the sync resumes.

3. **Check for Updates**:
   - Make sure you are using the latest version of CloudSync. You can check for updates in the application settings.

4. **Review Sync Settings**:
   - Open the settings in CloudSync and verify that the correct folders are selected for synchronization.

5. **Clear Cache** (if applicable):
   - If you are using a mobile or desktop app, clearing the cache may help. Look for this option in the settings.

6. **Log O

Notice how the general support agent acknowledges the user's frustration (empathetic_support module) while providing clear guidance.

In [16]:
# Test with technical support prompt
messages = [
    SystemMessage(content=technical_support_prompt),
    HumanMessage(content=user_query)
]

response = llm.invoke(messages)
print("=" * 60)
print("TECHNICAL SUPPORT AGENT RESPONSE")
print("=" * 60)
print(response.content)

TECHNICAL SUPPORT AGENT RESPONSE
I understand how frustrating it can be when file synchronization is not working as expected. Let's troubleshoot the issue with CloudSync together. Here are some steps you can follow to resolve the problem:

1. **Check Internet Connection**:
   - Ensure that your device is connected to the internet.
   - Try accessing a website to confirm connectivity.

2. **Restart CloudSync**:
   - Close the CloudSync application completely.
   - Reopen the application and check if the sync resumes.

3. **Check for Updates**:
   - Ensure that you are using the latest version of CloudSync.
   - If an update is available, install it and restart the application.

4. **Review Sync Settings**:
   - Open the CloudSync settings and verify that the correct folders are selected for synchronization.
   - Ensure that there are no conflicting settings that might prevent syncing.

5. **Check for File Conflicts**:
   - Look for any files that may be causing conflicts (e.g., files wi

The technical support agent focuses more on diagnostic steps and technical details, reflecting its different module composition.